In [11]:
# Module Importations
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [12]:
# Project Module Importations
from src.data import load_data
from src.data import split_data
from src.features import hyperparameter_opt
from src.models import model_evaluation

In [13]:
# Load dataset.
original_dataset = load_data.load_motor_data()

In [14]:
# Split data into training / evaluation sets
training_set, test_set = split_data.split_train_test(original_dataset, 0.8)

In [15]:
# Drop profile id data column (both sets)
training_set = training_set.drop("profile_id", axis = 1)
print(training_set.describe())

test_set = test_set.drop("profile_id", axis = 1)

ambient        coolant            u_d            u_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.005043       0.005308       0.003792      -0.005359   
std         0.996490       1.002488       0.998747       1.001464   
min        -5.239872      -1.270450      -1.654002      -1.852853   
25%        -0.601386      -1.037872      -0.831843      -0.924537   
50%         0.266671      -0.178200       0.267391      -0.098490   
75%         0.686838       0.673512       0.358587       0.848608   
max         2.954662       2.296845       2.273808       1.788773   

         motor_speed         torque            i_d            i_q  \
count  199614.000000  199614.000000  199614.000000  199614.000000   
mean       -0.004852      -0.001687       0.004184      -0.001570   
std         1.001954       0.998525       0.999021       0.998389   
min        -1.353747      -3.339106      -3.235659      -3.329749   
25%        -0.951892      -0.267419      -0.75

In [16]:
# Create rotor target
rotor_training_data = training_set.drop("pm", axis = 1)
rotor_label_data = training_set["pm"].copy()

rotor_test_data = test_set.drop("pm", axis = 1)
rotor_test_label_data = test_set["pm"].copy()

In [17]:
# Linear Regression Model (Target - Rotor Temperature)
lin_reg = LinearRegression()
lin_reg.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Evaluate on validation data

#mse_eval = mean_squared_error(rotor_test_label_data, rotor_pred_eval)
#rmse_eval = np.sqrt(mse_eval)
#mae_eval = mean_absolute_error(rotor_test_label_data, rotor_pred_eval)
#r2_eval = lin_reg.score(rotor_test_data, rotor_test_label_data)

# Print CV results
model_name = "Linear Regression"
print(model_name, "RMSE Mean (train):", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation (train):", scores_rmse.std())

print(model_name, "MAE Mean (train):", scores_mae.mean())
print(model_name, "MAE Standard Deviation (train):", scores_mae.std())

# Evaluation
rotor_pred_eval = lin_reg.predict(rotor_test_data)


#print("rmse Evaluation:", rmse_eval)
#print("mae Evaluation:", mae_eval)
#print("r2 Evaluation:", r2_eval)

Linear Regression Model RMSE Mean (train): 0.4749361664649533
Linear Regression Model RMSE Standard Deviation (train): 0.0021400886517142672
Linear Regression Model MAE Mean (train): 0.3614280996261001
Linear Regression Model MAE Standard Deviation (train): 0.0016560246676062818
rmse Evaluation: 0.4753533122550699
mae Evaluation: 0.36201495208919415
r2 Evaluation: 0.7718320737434564


In [18]:
# Polynomial Regression Model (Target - Rotor Temperature)
lin_reg_poly = LinearRegression()

# Create polynomial dataset
poly_features = PolynomialFeatures(degree = 2, include_bias = False)
rotor_training_poly = poly_features.fit_transform(rotor_training_data)

lin_reg_poly.fit(rotor_training_poly, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(lin_reg_poly, rotor_training_poly, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(lin_reg_poly, rotor_training_poly, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Evaluate on validation data
rotor_test_poly = poly_features.fit_transform(rotor_test_data)
rotor_pred_eval = lin_reg_poly.predict(rotor_test_poly)
mse_eval = mean_squared_error(rotor_test_label_data, rotor_pred_eval)
rmse_eval = np.sqrt(mse_eval)
mae_eval = mean_absolute_error(rotor_test_label_data, rotor_pred_eval)
r2_eval = lin_reg_poly.score(rotor_test_poly, rotor_test_label_data)

# Print results
model_name = "Linear Regression - Polynomial"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Std:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Std:", scores_mae.std())

print("rmse Evaluation:", rmse_eval)
print("mae Evaluation:", mae_eval)
print("r2 Evaluation:", r2_eval)

Linear Regression - Polynomial RMSE Mean: 0.41047276311126407
Linear Regression - Polynomial RMSE Std: 0.004807933470279054
Linear Regression - Polynomial MAE Mean: 0.30105790569514473
Linear Regression - Polynomial MAE Std: 0.0012163879552887916
rmse Evaluation: 0.4088304713024484
mae Evaluation: 0.30157356945924835
r2 Evaluation: 0.8312250214320183


In [19]:
# Decision Tree Regressor Model (Target - Rotor Temperature)
tree_regr = DecisionTreeRegressor()
tree_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(tree_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Evaluate on validation data
rotor_pred_eval = tree_regr.predict(rotor_test_data)
mse_eval = mean_squared_error(rotor_test_label_data, rotor_pred_eval)
rmse_eval = np.sqrt(mse_eval)
mae_eval = mean_absolute_error(rotor_test_label_data, rotor_pred_eval)
r2_eval = tree_regr.score(rotor_test_data, rotor_test_label_data)

# Print results
model_name = "Decision Tree Regressor"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

print("rmse Evaluation:", rmse_eval)
print("mae Evaluation:", mae_eval)
print("r2 Evaluation:", r2_eval)

Decision Tree Regressor RMSE Mean: 0.12499158062819371
Decision Tree Regressor RMSE Standard Deviation: 0.005620829891001395
Decision Tree Regressor MAE Mean: 0.029249208973060603
Decision Tree Regressor MAE Standard Deviation: 0.0012057811274619073
rmse Evaluation: 0.10758677478832664
mae Evaluation: 0.023559500162866433
r2 Evaluation: 0.9883120236679765


In [20]:
# Random Forest Regressor Model (Target - Rotor Temperature)
forest_regr = RandomForestRegressor()
forest_regr.fit(rotor_training_data, rotor_label_data)

# Compute RMSE via cross validation
scores_mse = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_squared_error", cv = 5)
scores_rmse = np.sqrt(-scores_mse)

# Compute MAE via cross validation
scores_mae = cross_val_score(forest_regr, rotor_training_data, rotor_label_data, scoring = "neg_mean_absolute_error", cv = 5)
scores_mae = -1 * scores_mae

# Evaluate on validation data
rotor_pred_eval = forest_regr.predict(rotor_test_data)
mse_eval = mean_squared_error(rotor_test_label_data, rotor_pred_eval)
rmse_eval = np.sqrt(mse_eval)
mae_eval = mean_absolute_error(rotor_test_label_data, rotor_pred_eval)
r2_eval = forest_regr.score(rotor_test_data, rotor_test_label_data)

# Print results
model_name = "Random Forest Regressor"
print(model_name, "RMSE Mean:", scores_rmse.mean())
print(model_name, "RMSE Standard Deviation:", scores_rmse.std())

print(model_name, "MAE Mean:", scores_mae.mean())
print(model_name, "MAE Standard Deviation:", scores_mae.std())

print("rmse Evaluation:", rmse_eval)
print("mae Evaluation:", mae_eval)
print("r2 Evaluation:", r2_eval)

Random Forest Regressor RMSE Mean: 0.07598667321870968
Random Forest Regressor RMSE Standard Deviation: 0.0020540376265633397
Random Forest Regressor MAE Mean: 0.02930183606381942
Random Forest Regressor MAE Standard Deviation: 0.0004773759954893715
rmse Evaluation: 0.06523092994576536
mae Evaluation: 0.024322107164163935
r2 Evaluation: 0.99570336277893


In [21]:
# Random Forest Regressor - Optimised Hyperparameters
forest_regr = RandomForestRegressor()

best_params, best_est, best_score = hyperparameter_opt.perform_gridsearch("Random Forest Regressor [Opt]", forest_regr, rotor_training_data, rotor_label_data)

# Evaluate on validation data
rotor_pred_eval = forest_regr.predict(rotor_test_data)
mse_eval = mean_squared_error(rotor_test_label_data, rotor_pred_eval)
rmse_eval = np.sqrt(mse_eval)
mae_eval = mean_absolute_error(rotor_test_label_data, rotor_pred_eval)
r2_eval = forest_regr .score(rotor_test_data, rotor_test_label_data)

print("rmse Evaluation:", rmse_eval)
print("mae Evaluation:", mae_eval)
print("r2 Evaluation:", r2_eval)

Random Forest Regressor [Opt] - Best Params: {'bootstrap': False, 'max_features': 4, 'n_estimators': 10}
Random Forest Regressor [Opt] - Best Estimator: RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=4, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
Random Forest Regressor [Opt] - Best Score: 0.07212845512150008


NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.